Ce notebook a pour but de
- prendre les données res_all_INC.csv
- sélectionner les exempels pour lesquels RL_NUM n'est pas manquant
- de supprimer les variables catégorielles ayant plus de 100 valeurs possibles
- de normaliser les donnnées
- d'imputer les donnés manquantes (différentes méthodes)
- exporter en csv

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv("/home/joon/math/herbier/res_all_INC.csv")

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,20,21,23,24,26,28,72,73,84,85,86,87,100,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On selectionne les lignes pour lesquelles RL_NUM n'est pas manquant et RL_STATUT est différent de DD

In [6]:
data1 = data[pd.notna(data['RL_NUM']) & (data['RL_STATUT']!='DD')]

In [7]:
columns_to_exclude = ['T1ID','SPECIESBASAUTHORS','SPECIESAUTHORS','INFRASPECIFICBASAUTHORS','INFRASPECIFICEPITHET','INFRASPECIFICAUTHORS','BASIONYMAUTHORS','DATEP','ACCEPTEDNAMEID','ACCEPTEDNAME','CLIMATE_AQUATIC','LC','MAJOR_CLIMATE','MAJOR_LIFEFORM','ANID','SUPPRESSED','LIFEFORM_BUDHGHT_MIN', "LIFEFORM_BUDHGHT_MAX", 'RL_NUM', 'RL_STATUT', 'YEAR_ASSESSED', 'LC', 'LCNT', 'RL_V1', 'RL_V2', 'RL_V3', 'RL', 'SONNERAT','pred_LC', 'p_LC', 'binf.p_LC', 'bsup.p_LC', 'p_rl_V3', 'p_LCNT', 'binf.p_LCNT', 'bsup.p_LCNT', 'miss']

data2 = data1.drop(columns=columns_to_exclude)

On marque les variables catégorielles comme telles

In [8]:
categorial_columns = ['CL','CLIMATE_ALTITUDE','CLIMATE_DRY','CLIMATE_TEMPERATE','CLIMATE_TROPICAL','LIFEFORM_CLIMBING','LIFEFORM_DEATH','LIFEFORM_ANNUAL','LIFEFORM_BIENNIAL','LIFEFORM_PERENNIAL','LIFEFORM_GEOPHYTE','LIFEFORM_HYDROPHYTE','LIFEFORM_HELOPHYTE','LIFEFORM_PHANEROPHYTE','LIFEFORM_NANOPHANEROPHYTE','LIFEFORM_HEMICRYPTOPHYTE','LIFEFORM_CHAMAEROPHYTE','LIFEFORM_EPIPHYTE','LIFEFORM_LITHOPHYTE','LIFEFORM_TUBER','LIFEFORM_RHIZOME','LIFEFORM_BULB','LIFEFORM_PSEUDOBULB','LIFEFORM_AQUATIC','LIFEFORM_SUCCULENT','LIFEFORM_CAUDIFORM','LIFEFORM_PARASITE','LIFEFORM_CAUDEX','LIFEFORM_PACHYCAUL','LIFEFORM_EXPOSED']

In [9]:
data2.replace('?',np.nan,inplace=True)
data2.replace('1',1.,inplace=True)
data2.replace('0',0.,inplace=True)
data2.replace('2',2.,inplace=True)

In [10]:
data2[categorial_columns]=data2[categorial_columns].astype('category')

In [11]:
# conversion object => category
object_columns = (data2.dtypes == 'object')
# index des colonnes categorielles
index_object_columns = object_columns.index[object_columns]
#conversion
data2[index_object_columns]=data2[index_object_columns].astype('category')

In [12]:
# nombres de categories par colonne
data2.describe(exclude=['float64','int64','bool']).loc['unique']

ORDER                           59
SUPER_ORDER                      8
ACCEPTEDFAMILY                 265
FAMILY                         308
FAMILY_CLEAN                   279
FULLNAME                     15824
RANK                             4
GENUS                         3322
SPECIES                       9824
CL                               2
CLIMATE_ALTITUDE                 2
CLIMATE_DRY                      2
CLIMATE_TEMPERATE                2
CLIMATE_TROPICAL                 2
LIFEFORM_CLIMBING                3
LIFEFORM_DEATH                   3
LIFEFORM_ANNUAL                  2
LIFEFORM_BIENNIAL                2
LIFEFORM_PERENNIAL               2
LIFEFORM_GEOPHYTE                2
LIFEFORM_HYDROPHYTE              2
LIFEFORM_HELOPHYTE               2
LIFEFORM_PHANEROPHYTE            2
LIFEFORM_NANOPHANEROPHYTE        2
LIFEFORM_HEMICRYPTOPHYTE         2
LIFEFORM_CHAMAEROPHYTE           2
LIFEFORM_EPIPHYTE                3
LIFEFORM_LITHOPHYTE              2
LIFEFORM_TUBER      

On repère les variables catégorielles avec un grand nombre de catégories pour les supprimer

In [13]:
high_cardidality_features_boolean = (data2.describe(exclude=['float64','int64','bool']).loc['unique'] > 100)
high_cardidality_features_list = list(high_cardidality_features_boolean.index[high_cardidality_features_boolean])

In [14]:
X_with_missing_and_categorical = data2.drop(columns=high_cardidality_features_list)
# X_with_missing_and_categorical = data2

On considère différentes sorties possible. Soit juste LC, soit RL_NUM. On exporte en csv.

In [15]:
y_LC = data1['LC'].astype('int')
y_RL_NUM = data1['RL_NUM'].astype('int')

y_LC.to_csv(path_or_buf='y_LC.csv',header=False,index=False)
y_RL_NUM.to_csv(path_or_buf='y_RL_NUM.csv',header=False,index=False)

On procède au one-hot encoding des variables catégorielles

In [16]:
X_with_missing = pd.get_dummies(X_with_missing_and_categorical)

On normalise les données

In [17]:
from sklearn.preprocessing import StandardScaler
X_with_missing = StandardScaler().fit(X_with_missing).transform(X_with_missing)

/usr/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


On impute les données manquantes par différents algos.

In [2]:
from fancyimpute import KNN, SimpleFill, SoftImpute, IterativeImputer, BiScaler, IterativeSVD, NuclearNormMinimization, MatrixFactorization

Using TensorFlow backend.


In [56]:
X_SimpleFill = SimpleFill().fit_transform(X_with_missing)
np.savetxt('X_SimpleFill.csv', X_SimpleFill, delimiter=',')

In [57]:
X_5NN = KNN(k=5).fit_transform(X_with_missing)
np.savetxt('X_5NN.csv', X_5NN, delimiter=',')

Imputing row 1/15824 with 11 missing, elapsed time: 162.220
Imputing row 101/15824 with 33 missing, elapsed time: 162.354


Imputing row 201/15824 with 8 missing, elapsed time: 162.484
Imputing row 301/15824 with 14 missing, elapsed time: 162.605


Imputing row 401/15824 with 33 missing, elapsed time: 162.734
Imputing row 501/15824 with 33 missing, elapsed time: 162.871


Imputing row 601/15824 with 8 missing, elapsed time: 163.023
Imputing row 701/15824 with 26 missing, elapsed time: 163.154


Imputing row 801/15824 with 6 missing, elapsed time: 163.276
Imputing row 901/15824 with 0 missing, elapsed time: 163.389
Imputing row 1001/15824 with 25 missing, elapsed time: 163.471


...

Imputing row 14901/15824 with 37 missing, elapsed time: 179.635
Imputing row 15001/15824 with 37 missing, elapsed time: 179.804


Imputing row 15101/15824 with 18 missing, elapsed time: 179.913
Imputing row 15201/15824 with 8 missing, elapsed time: 180.006


Imputing row 15301/15824 with 33 missing, elapsed time: 180.122
Imputing row 15401/15824 with 14 missing, elapsed time: 180.266


Imputing row 15501/15824 with 33 missing, elapsed time: 180.407
Imputing row 15601/15824 with 33 missing, elapsed time: 180.555


Imputing row 15701/15824 with 33 missing, elapsed time: 180.665
Imputing row 15801/15824 with 11 missing, elapsed time: 180.809


In [61]:
X_SoftImpute = SoftImpute().fit_transform(X_with_missing)
np.savetxt('X_SoftImpute.csv', X_SoftImpute, delimiter=',')

[SoftImpute] Max Singular Value of X_init = 685.909398


[SoftImpute] Iter 1: observed MAE=0.029928 rank=128


[SoftImpute] Iter 2: observed MAE=0.029944 rank=127


[SoftImpute] Iter 3: observed MAE=0.029953 rank=127


[SoftImpute] Iter 4: observed MAE=0.029960 rank=127


...

[SoftImpute] Iter 62: observed MAE=0.030012 rank=127


[SoftImpute] Iter 63: observed MAE=0.030012 rank=127


[SoftImpute] Iter 64: observed MAE=0.030012 rank=127


[SoftImpute] Iter 65: observed MAE=0.030012 rank=127


[SoftImpute] Iter 66: observed MAE=0.030012 rank=127
[SoftImpute] Stopped after iteration 66 for lambda=13.718188


In [64]:
X_IterativeSVD = IterativeSVD().fit_transform(X_with_missing)
np.savetxt('X_IterativeSVD.csv', X_IterativeSVD, delimiter=',')

[IterativeSVD] Iter 1: observed MAE=0.246206


[IterativeSVD] Iter 2: observed MAE=0.239842


[IterativeSVD] Iter 3: observed MAE=0.226978


[IterativeSVD] Iter 4: observed MAE=0.224976


[IterativeSVD] Iter 5: observed MAE=0.231567


...

[IterativeSVD] Iter 145: observed MAE=0.219992


[IterativeSVD] Iter 146: observed MAE=0.219983


[IterativeSVD] Iter 147: observed MAE=0.219975


[IterativeSVD] Iter 148: observed MAE=0.219966


[IterativeSVD] Iter 149: observed MAE=0.219958


In [65]:
X_IterativeImputer = IterativeImputer().fit_transform(X_with_missing)
np.savetxt('X_IterativeImputer.csv', X_IterativeImputer, delimiter=',')

In [71]:
X_MatrixFactorization = MatrixFactorization().fit_transform(X_with_missing)
np.savetxt('X_MatrixFactorization.csv', X_MatrixFactorization, delimiter=',')

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Train on 2205984 samples, validate on 245110 samples
Epoch 1/10000


2205984/2205984 [==============================] - 1s 0us/step - loss: 1.2533 - val_loss: 1.2763


Epoch 2/10000


...

2205984/2205984 [==============================] - 2s 1us/step - loss: 0.7821 - val_loss: 0.8476


Epoch 605/10000


2205984/2205984 [==============================] - 2s 1us/step - loss: 0.7818 - val_loss: 0.8474


Epoch 606/10000


2205984/2205984 [==============================] - 2s 1us/step - loss: 0.7815 - val_loss: 0.8472


In [18]:
X_BiScaler = BiScaler().fit_transform(X_with_missing)
np.savetxt('X_BiScaler.csv', X_BiScaler, delimiter=',')

[BiScaler] Initial log residual value = 9.862508
[BiScaler] Iter 1: log residual = 7.992389, log improvement ratio=1.870119


[BiScaler] Iter 2: log residual = 7.290234, log improvement ratio=0.702155
[BiScaler] Iter 3: log residual = 7.013815, log improvement ratio=0.276419


[BiScaler] Iter 4: log residual = 6.811306, log improvement ratio=0.202510
[BiScaler] Iter 5: log residual = 6.655133, log improvement ratio=0.156173


[BiScaler] Iter 6: log residual = 6.530467, log improvement ratio=0.124666
[BiScaler] Iter 7: log residual = 6.428451, log improvement ratio=0.102016


[BiScaler] Iter 8: log residual = 6.343501, log improvement ratio=0.084950
[BiScaler] Iter 9: log residual = 6.271869, log improvement ratio=0.071632


...

[BiScaler] Iter 48: log residual = 5.750285, log improvement ratio=0.001568
[BiScaler] Iter 49: log residual = 5.748800, log improvement ratio=0.001485


[BiScaler] Iter 50: log residual = 5.747393, log improvement ratio=0.001408
[BiScaler] Iter 51: log residual = 5.746056, log improvement ratio=0.001337


[BiScaler] Iter 52: log residual = 5.744785, log improvement ratio=0.001270
[BiScaler] Iter 53: log residual = 5.743577, log improvement ratio=0.001208


[BiScaler] Iter 54: log residual = 5.742427, log improvement ratio=0.001150
[BiScaler] Iter 55: log residual = 5.741332, log improvement ratio=0.001095


[BiScaler] Iter 56: log residual = 5.740288, log improvement ratio=0.001043
[BiScaler] Iter 57: log residual = 5.739294, log improvement ratio=0.000994


Comme indiqué dans la documentation, la méthode suivante est trop lente (pour de grandes matrices)

In [ ]:
X_NuclearNormMinimization = NuclearNormMinimization().fit_transform(X_with_missing)
np.savetxt('X_NuclearNormMinimization.csv', X_NuclearNormMinimization, delimiter=',')